In [1]:
__author__ = 'tkurth'
import sys
import os
import numpy as np
import scipy as sp
import pandas as pd
import h5py
from matplotlib import pyplot as plt
import matplotlib.mlab as mlab
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from nbfinder import NotebookFinder
sys.meta_path.append(NotebookFinder())
%matplotlib inline
import time
from tqdm import tqdm
import re
import pickle

/global/common/cori/software/python/3.5-anaconda/envs/deeplearning/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Load Data

In [2]:
prefix='output'

#retrieve files
directory="/global/cscratch1/sd/tkurth/atlas_dl/atlas_caffe/hero_eval"
filelist=[x for x in os.listdir(directory) if x.startswith(prefix+'_chunk')]

In [3]:
filelist

['output_chunk24.h5',
 'output_chunk107.h5',
 'output_chunk50.h5',
 'output_chunk55.h5',
 'output_chunk25.h5',
 'output_chunk74.h5',
 'output_chunk115.h5',
 'output_chunk106.h5',
 'output_chunk18.h5',
 'output_chunk40.h5',
 'output_chunk47.h5',
 'output_chunk92.h5',
 'output_chunk19.h5',
 'output_chunk122.h5',
 'output_chunk5.h5',
 'output_chunk1.h5',
 'output_chunk14.h5',
 'output_chunk44.h5',
 'output_chunk127.h5',
 'output_chunk41.h5',
 'output_chunk88.h5',
 'output_chunk49.h5',
 'output_chunk99.h5',
 'output_chunk10.h5',
 'output_chunk34.h5',
 'output_chunk64.h5',
 'output_chunk59.h5',
 'output_chunk35.h5',
 'output_chunk121.h5',
 'output_chunk11.h5',
 'output_chunk31.h5',
 'output_chunk81.h5',
 'output_chunk109.h5',
 'output_chunk57.h5',
 'output_chunk21.h5',
 'output_chunk98.h5',
 'output_chunk48.h5',
 'output_chunk45.h5',
 'output_chunk86.h5',
 'output_chunk69.h5',
 'output_chunk97.h5',
 'output_chunk39.h5',
 'output_chunk82.h5',
 'output_chunk42.h5',
 'output_chunk62.h5',
 'out

## Populate Arrays

In [4]:
datalist=[]

for fname in tqdm(filelist):
    
    #determine chunkid:
    chunkid=int(re.match(prefix+r'_chunk(.*?).h5',fname).groups()[0])
    
    #print "Open file "+fname
    f = h5py.File(directory+'/'+fname,'r')
    
    tmpdata={}
    for item in f.items():
        
        #determine name and ID of item
        itemname=item[0].split('_')[0]
        itemid=int(item[0].split('_')[1])
        
        #read data
        data=list(f[item[0]].value)
        
        #add to dictionary
        if (chunkid,itemid) not in tmpdata.keys():
            tmpdata[(chunkid,itemid)]={itemname: data}
        else:
            tmpdata[(chunkid,itemid)][itemname]=data
    
    #close the file
    f.close()
    
    #put in list:
    for item in tmpdata:
        dct=tmpdata[item].copy()
        dct['chunk_id']=item[0]
        dct['item_id']=item[1]
        datalist.append(dct)

100%|██████████| 128/128 [03:55<00:00,  1.85s/it]


In [5]:
#Convert to dataframe:
dflist=[]
for item in tqdm(datalist):
    datareclist=[]
    for idx in range(len(item['softmax'])):
        tmpdict={'signal_prob': item['softmax'][idx][1],
                 'label': item['label'][idx],
                 'weight': item['weight'][idx],
                 'psr': item['psr'][idx],
                 'jz': item['jz'][idx],
                 'mg': item['mg'][idx],
                 'mn': item['mn'][idx]
                }
        datareclist.append(tmpdict)
    dflist.append(pd.DataFrame(datareclist))

#convert to dataframe
datadf=pd.concat(dflist)

100%|██████████| 62720/62720 [00:41<00:00, 1505.88it/s]


## ROC

In [6]:
#seldf=datadf.ix[ ((datadf.mg==1400) & (datadf.mg==850)) | (datadf.jz!=0) ]
seldf=datadf

In [7]:
#compute ROC from data
fpr, tpr, thresholds = metrics.roc_curve(datadf.label, datadf.signal_prob, pos_label=1, sample_weight=datadf.weight)
fpr_cut, tpr_cut, thresholds = metrics.roc_curve(datadf.label, datadf.psr, pos_label=1, sample_weight=datadf.weight)
fpr_cut=fpr_cut[1]
tpr_cut=tpr_cut[1]

In [8]:
#plot the data
plt.figure()
lw = 2
#full curve
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % metrics.auc(fpr,tpr,reorder=True))
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.scatter([fpr_cut],[tpr_cut], color='dodgerblue', label='standard cuts')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.savefig('plots/ROC_1400_850.png',dpi=300)
#plt.show()

#zoomed-in
#plt.plot(fpr, tpr, color='darkorange',
#         lw=lw, label='ROC curve (area = %0.2f)' % metrics.auc(fpr,tpr))
#plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 0.0004])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="upper right")
plt.savefig('plots/ROC_1400_850_zoom.png',dpi=300)
plt.show()

In [11]:
#directly compare at the PSR FPR:
eff=np.interp([fpr_cut],fpr,tpr,0.1)[0]
eff_ratio=eff/tpr_cut
print("Efficiency: ",eff*100,"%")
print("Improvement in efficiency: ",eff_ratio)
print("Benchmark false positive rate:",fpr_cut)
print("Benchmark true positive rate:",tpr_cut)

Efficiency:  54.5482308151 %
Improvement in efficiency:  1.30650526653
Benchmark false positive rate: 0.000235635916156
Benchmark true positive rate: 0.417512521476


# CNN results

In [ ]:
#threshold
threshold=0.999

In [ ]:
#compute signal efficiency:
total_signal=np.sum(l_data)
found_signal=np.sum([1. if x[0]*x[1]>threshold else 0. for x in zip(l_data,p_data)])
print 'sig-efficiency(CNN): ',found_signal/total_signal
print 'sig-survivors(CNN): ',found_signal

#compute bg rejection:
total_background=np.sum([1.-x for x in l_data])
false_positive_bg=np.sum([1. if (1.-x[0])*x[1]>threshold else 0. for x in zip(l_data,p_data)])
print 'bg-rejection(CNN): ',1.-false_positive_bg/total_background
print 'bg-survivors(CNN): ',(1.-(1.-false_positive_bg/total_background))*total_background

In [ ]:
#AMS
#compute s:
sval=np.sum([x[2] if x[0]*x[1]>threshold else 0. for x in zip(l_data,p_data,w_data)])
bval=np.sum([x[2] if (1.-x[0])*x[1]>threshold else 0. for x in zip(l_data,p_data,w_data)])
brval=10.

#print AMS results
print "AMS(CNN) = ",np.sqrt(2.*((sval+bval+brval)*np.log(1.+sval/(bval+brval))-sval))

# Cut-based results

In [ ]:
#compute signal efficiency:
total_signal=np.sum(l_data)
found_signal=np.sum([1. if x[0]*x[1]>0. else 0. for x in zip(l_data,c_data)])
print 'sig-efficiency(CUT): ',found_signal/total_signal
print 'sig-survivors(CUT): ',found_signal

#compute bg rejection:
total_background=np.sum([1.-x for x in l_data])
false_positive_bg=np.sum([1. if (1.-x[0])*x[1]>0. else 0. for x in zip(l_data,c_data)])
print 'bg-rejection(CUT): ',1.-false_positive_bg/total_background
print 'bg-survivors(CUT): ',(1.-(1.-false_positive_bg/total_background))*total_background

In [ ]:
#AMS
#compute s:
sval=np.sum([x[2] if x[0]*x[1]>threshold else 0. for x in zip(l_data,c_data,w_data)])
bval=np.sum([x[2] if (1.-x[0])*x[1]>threshold else 0. for x in zip(l_data,c_data,w_data)])
brval=10.

#print AMS results
print "AMS(CUT) = ",np.sqrt(2.*((sval+bval+brval)*np.log(1.+sval/(bval+brval))-sval))